In [ ]:
from dotenv import load_dotenv
import os
import csv
from langchain.llms import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI

load_dotenv()
openai_api_key = os.environ['OPENAI_API_KEY']

llm = OpenAI(temperature=0.9)

In [ ]:
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'
embedding = OpenAIEmbeddings()
docsearch = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [ ]:
query = "For what temperature can i use a brass valve with fkm seal?"
docs = docsearch.similarity_search(query)

chain = load_qa_with_sources_chain(OpenAI(temperature=1), chain_type="stuff")
chain({"input_documents": docs, "question": query}, return_only_outputs=True)